In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
#Se carga el archivo de restaurantes ya limpio para poder obtener las ids de google
# y asi crear un data set solo las reviews de los restaurantes
df_restaurantes= pd.read_parquet('Datos limpios/restaurantes_clean.parquet')
gmaps=df_restaurantes['gmap_id'].astype(str).tolist()

In [3]:
#Hago un diccionarion con las rutas de los archivos de las reviews
ruta_review_estados={
    'NY': 'Datasets/Google Maps/reviews-estados/review-New_York',
    'CA': 'Datasets/Google Maps/reviews-estados/review-California',
    'TX': 'Datasets/Google Maps/reviews-estados/review-Texas',
    'FL': 'Datasets/Google Maps/reviews-estados/review-Florida',
    'PA': 'Datasets/Google Maps/reviews-estados/review-Pennsylvania',

}

In [6]:
reviews={}
#Recorro todos los archivos y solo me quedo con las reviews de los restaurantes
for ruta in ruta_review_estados.values():
#Por cada ruta se recorren todos los archvivos
    for archivo in os.listdir(ruta):
        ruta_archivo=os.path.join(ruta, archivo)
        with open(ruta+'/'+archivo,'r') as file:
            for linea in file:
                data = json.loads(linea)
                #Si la review es de un restaurante se guarda en el diccionario
                if data['gmap_id'] in gmaps:
                    reviews[data['gmap_id']]=data      
            
       

In [7]:
#Ahora transformamos el diccionario en un data frame
df_reviews=pd.DataFrame(reviews).T

In [8]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4612 entries, 0x89c261f60bdf13db:0x38da730e4687a97b to 0x89c6c7f7f02bfe15:0xa5ca7c2c7bd545e
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  4612 non-null   object
 1   name     4612 non-null   object
 2   time     4612 non-null   object
 3   rating   4612 non-null   object
 4   text     139 non-null    object
 5   pics     69 non-null     object
 6   resp     459 non-null    object
 7   gmap_id  4612 non-null   object
dtypes: object(8)
memory usage: 324.3+ KB


In [13]:
#Eliminamos las columnas que no nos interesan
df_reviews.drop(columns=['pics','time','resp'],inplace=True)

In [17]:
#Unimos los dos dataframes
df_reviews=df_reviews.merge(df_restaurantes[['gmap_id','city','state']],on='gmap_id',how='left')

In [20]:
#Reordenamos las columnas 
df_reviews=df_reviews[['gmap_id','name','city','state','rating','text']]

In [21]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4612 entries, 0 to 4611
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  4612 non-null   object
 1   name     4612 non-null   object
 2   city     4612 non-null   object
 3   state    4612 non-null   object
 4   rating   4612 non-null   object
 5   text     139 non-null    object
dtypes: object(6)
memory usage: 216.3+ KB


In [22]:
df_reviews.head()

,gmap_id,name,city,state,rating,text
0,0x89c261f60bdf13db:0x38da730e4687a97b,SK,Queens,NY,1,Staff needs to pay attention to what costumers...
1,0x89c258ffaeaba947:0x8355860772a595a9,Matt Schaffnit,New York,NY,5,Used to get lunch here nearly every day. You w...
2,0x89c2f5c3caef3fb5:0xb7f855503b4bc974,Rafiat Aromire,Bronx,NY,4,"Good food, clean place !"
3,0x89c25bb23e0eec35:0xe8a6498dea29132b,Chris,Brooklyn,NY,1,None
4,0x89c26321a3efcfe9:0x962f89abf63762db,Princess Isabell,Franklin Square,NY,5,Great place


In [23]:
df_reviews.to_parquet('Datos limpios/reviews_restaurantes.parquet')